# Test of Structural Mechanics
We will now test the python interface of teh structural mechanics part.
For that we will use the test `test/test_model/test_structural_mechanics_model/test_structural_mechanics_model_bernoulli_beam_2.cc`, which we will simply reproduce.


In [1]:
import py11_akantu as pyaka

### Creating the Mesh

In [2]:
# Create a mesh for the two dimensional case
beam = pyaka.Mesh(2)

In [3]:
# read in the mesh description
beam.read("_bernoulli_beam_2.msh", pyaka.MeshIOType._miot_gmsh_struct)

### Creating the Model

In [4]:
model = pyaka.StructuralMechanicsModel(beam)

#### Setting up the Modell

##### Creating and Inserting the Materials

In [6]:
mat1 = pyaka.StructuralMaterial()
mat1.E = 3e10;
mat1.I = 0.0025;
mat1.A = 0.01;
model.addMaterial(mat1)

0

In [7]:
mat2 = pyaka.StructuralMaterial()
mat2.E = 3e10;
mat2.I = 0.00128;
mat2.A = 0.01;
model.addMaterial(mat2)

1

##### Initializing the Model

In [8]:
#model.initFull(pyaka.AnalysisMethod._static)
model.initFull()

##### Assigning the Materials

In [10]:
materials = model.getElementMaterialMap(pyaka.ElementType._bernoulli_beam_2)

In [11]:
materials

array([[1693225798],
       [     21951]], dtype=uint32)

Once we have written to the `materials` variable, everything becomes unstable.
And the kernel will die.

In [ ]:
materials[0][0] = 0
materials[1][0] = 1

In [ ]:
materials

##### Setting Boundaries

In [ ]:
M = 3600.
q = -6000.
L = 10.

In [ ]:
forces = model.getExternalForce()
forces

In [ ]:
# Neumann
forces = model.getExternalForce()
forces[2, 2] = -M
forces[0, 1] =  q * L / 2
forces[0, 2] =  q * L * L / 12
forces[1, 1] =  q * L / 2
forces[1, 2] = -q * L * L / 12
forces

In [ ]:
# Dirichlets
boundary = model.getBlockedDOFs()
boundary[0, :] = True
boundary[1, :] = False
boundary[2, :] = False
boundary[2, 1] = True

In [ ]:
model.getExternalForce()

### Solving the System

In [ ]:
model.solveStep()

In [ ]:
disp = model.getDisplacement()

In [ ]:
d1 = disp[1, 2]
d2 = disp[2, 2]
d3 = disp[1, 0]

In [ ]:
d1, 5.6 / 4800